In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import HousePriceData, TestingData
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

classifier = LinearRegression()

principle_components = 45
threshold = 0.04

x_scaler = Pipeline([
('scaler', StandardScaler()),
('pca', PCA(n_components=principle_components)),
])
output_scaler = StandardScaler()

In [3]:
data = HousePriceData("dataset/train.csv", x_scaler, output_scaler)

In [6]:
data.get_dataframe()[["GarageYrBlt","YearBuilt"]]

,GarageYrBlt,YearBuilt
0,2003.0,2003
1,1976.0,1976
2,2001.0,2001
3,1998.0,1915
4,2000.0,2000
...,...,...
1455,1999.0,1999
1456,1978.0,1978
1457,1941.0,1941
1458,1950.0,1950


In [16]:
null_cols = data.check_na()

LotFrontage 259
Alley 1369
MasVnrType 8
MasVnrArea 8
BsmtQual 37
BsmtCond 37
BsmtExposure 38
BsmtFinType1 37
BsmtFinType2 38
Electrical 1
FireplaceQu 690
GarageType 81
GarageYrBlt 81
GarageFinish 81
GarageQual 81
GarageCond 81
PoolQC 1453
Fence 1179
MiscFeature 1406


In [17]:
replace_by = [
    "M", 
    "NA",
    "None",          #Can be changed to NonewNA
    "M",             #Can be changed to ZwM
    "NA",
    "NA",
    "NA",            #Includes option "No" but mean different thing
    "NA",
    "NA",
    "SBrkr",            #Electrical, only 1 NA
    "NA",
    "NA",
    "M",             #Can be changed to Z
    "NA",
    "NA",
    "NA",
    "NA",
    "NA",
    "NA"
    ]

data.clean_data(null_cols, replace_by)

In [18]:
data.check_na()

[]

In [19]:
mapping = data.convert_to_categorical()

/Users/omkarmasur/Study/Masters/CSCI 567 - Machine Learning/Project Code/utils.py:112: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.__df["MSZoning"] = self.__df["MSZoning"].replace(self.msZoning_map)
/Users/omkarmasur/Study/Masters/CSCI 567 - Machine Learning/Project Code/utils.py:115: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.__df["Utilities"] = self.__df["Utilities"].replace(self.utilites_map)
/Users/omkarmasur/Study/Masters/CSCI 567 - Machine Learning/Project Code/utils.py:118: FutureWarning: Downcasting behavior

In [20]:
data.get_dataframe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,5,5,65.0,8450,1,1,3,3,0,4,...,0,4,4,5,0,2,2008,0,4,208500
1,0,5,80.0,9600,1,1,3,3,0,2,...,0,4,4,5,0,5,2007,0,4,181500
2,5,5,68.0,11250,1,1,0,3,0,4,...,0,4,4,5,0,9,2008,0,4,223500
3,6,5,60.0,9550,1,1,0,3,0,0,...,0,4,4,5,0,2,2006,0,0,140000
4,5,5,84.0,14260,1,1,0,3,0,2,...,0,4,4,5,0,12,2008,0,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5,5,62.0,7917,1,1,3,3,0,4,...,0,4,4,5,0,8,2007,0,4,175000
1456,0,5,85.0,13175,1,1,3,3,0,4,...,0,4,2,5,0,2,2010,0,4,210000
1457,6,5,66.0,9042,1,1,3,3,0,4,...,0,4,0,3,2500,5,2010,0,4,266500
1458,0,5,68.0,9717,1,1,3,3,0,4,...,0,4,4,5,0,4,2010,0,4,142125


In [22]:
selected_columns = data.get_columns_to_drop(threshold=threshold)

In [23]:
selected_columns

['LandContour',
 'Utilities',
 'Condition2',
 'BsmtFinSF2',
 'LowQualFinSF',
 'BsmtHalfBath',
 'MiscVal',
 'YrSold']

In [24]:
for column in selected_columns:
    data.drop_column(column)

In [25]:
data.fit_preprocessors()

In [26]:
X_train, X_test, y_train, y_test = data.get_train_test_split()

In [27]:
classifier.fit(X_train, y_train)

LinearRegression()

In [28]:
classifier.score(X_train, y_train)

0.8232316805136582

In [29]:
classifier.score(X_test, y_test)

0.8459213147256706

In [511]:
data.save_mapping("mapping.pickle")

In [512]:
data.get_dataframe()

,MSZoning,LotFrontage,LotArea,LotShape,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolQC,Fence,SaleType,SaleCondition,SalePrice
0,5,65.0,8450,3,5,5,7,2003,2003,1,...,2,0,61,0,0,4,4,0,4,208500
1,5,80.0,9600,3,24,2,6,1976,1976,1,...,2,298,0,0,0,4,4,0,4,181500
2,5,68.0,11250,0,5,5,7,2001,2002,1,...,2,0,42,0,0,4,4,0,4,223500
3,5,60.0,9550,0,6,5,7,1915,1970,1,...,2,0,35,272,0,4,4,0,0,140000
4,5,84.0,14260,0,15,5,8,2000,2000,1,...,2,192,84,0,0,4,4,0,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5,62.0,7917,3,8,5,6,1999,2000,1,...,2,0,40,0,0,4,4,0,4,175000
1456,5,85.0,13175,3,14,2,6,1978,1988,1,...,2,349,0,0,0,4,2,0,4,210000
1457,5,66.0,9042,3,6,5,7,1941,2006,1,...,2,0,60,0,0,4,0,0,4,266500
1458,5,68.0,9717,3,12,2,5,1950,1996,3,...,2,366,0,112,0,4,4,0,4,142125


In [513]:
test_data = TestingData("mapping.pickle", "dataset/test.csv", data.get_dropped_columns(), data.get_processors()[0])

In [514]:
test_data.preprocess_data()

/Users/omkarmasur/Study/Masters/CSCI 567 - Machine Learning/Project Code/utils.py:237: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.__df[mapping.name] = self.__df[mapping.name].replace(mapping.mapping)


In [515]:
@test_data.form_test_file
def predict():
    predictions = classifier.predict(test_data.get_x())
    predictions = output_scaler.inverse_transform(predictions.reshape(-1, 1))
    return predictions

In [516]:
predict()

/Users/omkarmasur/Study/Masters/CSCI 567 - Machine Learning/Project Code/env/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
